In [71]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd
import tkinter as tk
from tkinter import filedialog
import pandas_ta as ta
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score
import seaborn as sns

root = tk.Tk()
root.wm_attributes('-topmost', 1)
root.withdraw()

# feature engineering and scaling
def prep(dataset):
    scaler = StandardScaler()

    macro_path1 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\IRP DATASET  - Copy of US_macroeconomics.csv"
    macro_path2 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\^VIX.csv"
    macro_path3 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\CORESTICKM159SFRBATL.csv"
    macro_path4 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\FFR.csv"
    macro_path5 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\GDP.csv"
    macro_path6 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\VIX9D_History.csv"
    macro_path7 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\GVZ_History.csv"
    macro_path8 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\OVX_History.csv"
    macro_path9 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\VVIX_History.csv"
    macro_data1 = pd.read_csv(macro_path1)
    macro_data2 = pd.read_csv(macro_path2)
    macro_data3 = pd.read_csv(macro_path3)
    macro_data4 = pd.read_csv(macro_path4)
    macro_data5 = pd.read_csv(macro_path5)
    macro_data6 = pd.read_csv(macro_path6)
    macro_data7 = pd.read_csv(macro_path7)
    macro_data8 = pd.read_csv(macro_path8)
    macro_data9 = pd.read_csv(macro_path9)

    dataset['Date'] = pd.to_datetime(dataset['Date'], dayfirst=True)
    dataset.set_index('Date', inplace=True)
    dataset['Target'] = np.where(dataset['Close'].shift(-1) > dataset['Close'], 1, 0)

    macro_data1['date'] = pd.to_datetime(macro_data1['date'], dayfirst=True)
    macro_data1.set_index('date', inplace=True)

    macro_data2['Date'] = pd.to_datetime(macro_data2['Date'], dayfirst=True)
    macro_data2.set_index('Date', inplace=True)

    macro_data3['Date'] = pd.to_datetime(macro_data3['Date'], dayfirst=True)
    macro_data3.set_index('Date', inplace=True)

    macro_data4['Date'] = pd.to_datetime(macro_data4['Date'], dayfirst=True)
    macro_data4.set_index('Date', inplace=True)

    macro_data5['Date'] = pd.to_datetime(macro_data5['Date'], dayfirst=True)
    macro_data5.set_index('Date', inplace=True)

    macro_data7['Date'] = pd.to_datetime(macro_data7['Date'], dayfirst=False)
    macro_data7.set_index('Date', inplace=True)

    macro_data8['Date'] = pd.to_datetime(macro_data8['Date'], dayfirst=False)
    macro_data8.set_index('Date', inplace=True)

    macro_data9['Date'] = pd.to_datetime(macro_data9['Date'])
    macro_data9.set_index('Date', inplace=True)

    merge_data1 = dataset.join(macro_data1, how='left').fillna(method='ffill')
    merge_data2 = merge_data1.join(macro_data2, how='left').fillna(method='ffill')
    merge_data3 = merge_data2.join(macro_data3, how='left').fillna(method='ffill')
    merge_data4 = merge_data3.join(macro_data4, how='left').fillna(method='ffill')
    merge_data5 = merge_data4.join(macro_data5, how='left').fillna(method='ffill')
    merge_data7 = merge_data5.join(macro_data7, how='left').fillna(method='ffill')
    merge_data8 = merge_data7.join(macro_data8, how='left').fillna(method='ffill')
    merge_final = merge_data8.join(macro_data9, how='left').fillna(method='ffill')

    merge_final['RSI (14D)'] = ta.rsi(merge_final['Close'], length=14)
    merge_final['20 Day CCI'] = ta.cci(high=merge_final['High'], low=merge_final['Low'], 
                                    close=merge_final['Close'], length=20)
    merge_final['Williams %R'] = ta.willr(high=merge_final['High'], low=merge_final['Low'], 
                                        close=merge_final['Close'], length=14)
    merge_final['EMA (5D)'] = merge_final['Close'].ewm(span=5, adjust=False).mean()
    merge_final['MA50'] = merge_final['Close'].rolling(window=50).mean()

    final_dataset = merge_final.dropna()

    time_shifted = final_dataset.resample('W-FRI').asfreq().dropna()

    features = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'CPI', 
                'Mortgage_rate', 'Unemp_rate', 'disposable_income','GVZ', 'OVX', 'VVIX', 
                'RSI (14D)', '20 Day CCI', 'Williams %R', 'EMA (5D)', 'MA50', 'DFF']
    
    time_shifted[features] = time_shifted[features].astype(float)
    time_shifted[features] = scaler.fit_transform(time_shifted[features])

    return time_shifted, scaler

# get dataset
file_path = filedialog.askopenfilename(parent=root, title="Select A File")
ticker = pd.read_csv(file_path)
ticker, scaler = prep(ticker)

ticker

,Open,High,Low,Close,Adj Close,Volume,Target,CPI,Mortgage_rate,Unemp_rate,...,DFF,GDP,GVZ,OVX,VVIX,RSI (14D),20 Day CCI,Williams %R,EMA (5D),MA50
Date,,,,,,,,,,,,,,,,,,,,,
2009-09-18,-1.474952,-1.473493,-1.456210,-1.465607,-1.465607,1.387820,0.0,-1.683557,1.915242,1.500431,...,-0.529865,14448.882,1.055772,0.351776,-1.466219,1.503123,1.100795,0.890461,-1.487862,-1.675514
2009-09-25,-1.529658,-1.535451,-1.531723,-1.541050,-1.541050,0.463427,1.0,-1.683557,1.915242,1.500431,...,-0.571081,14448.882,0.888946,0.510451,-1.384928,-0.095944,-0.109934,-0.681062,-1.512220,-1.632186
2009-10-02,-1.586074,-1.599389,-1.595973,-1.599146,-1.599146,1.367055,1.0,-1.640113,1.732344,1.588885,...,-0.571081,14651.249,0.806538,0.568428,-1.120584,-0.971968,-1.444447,-1.686548,-1.555907,-1.605224
2009-10-09,-1.479500,-1.483432,-1.465126,-1.466146,-1.466146,-0.160720,1.0,-1.640113,1.732344,1.588885,...,-0.584819,14651.249,1.170340,0.469562,-1.758262,0.576109,0.512426,0.644455,-1.498158,-1.581963
2009-10-16,-1.395998,-1.409957,-1.416937,-1.419954,-1.419954,0.788931,1.0,-1.640113,1.732344,1.588885,...,-0.584819,14651.249,0.681921,0.061280,-1.498735,0.748176,0.942693,0.786190,-1.429216,-1.558301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-19,2.481198,2.454110,2.444340,2.418291,2.418291,0.250222,0.0,2.517552,-1.534690,-0.976289,...,-0.639773,24654.603,-0.276827,0.074706,1.144107,-0.779904,-1.336500,-1.899822,2.488144,2.432283
2021-11-26,2.449547,2.422597,2.280393,2.282122,2.282122,-0.367350,1.0,2.517552,-1.534690,-0.976289,...,-0.639773,24654.603,0.125164,1.819517,3.311851,-1.813518,-2.300672,-1.663731,2.400766,2.442479
2021-12-03,2.229923,2.233803,2.152636,2.174075,2.174075,1.078900,1.0,2.624955,-1.481814,-1.108970,...,-0.639773,24654.603,0.111094,1.748113,3.694819,-1.610778,-1.475791,-1.417231,2.217995,2.444292


In [73]:
# create LSTM model class
class LSTM_Model(nn.Module):
    def __init__(self, input_layer, hidden_layer, output_layer, num_layers=4, dropout=0.3):
        super(LSTM_Model, self).__init__()
        self.hidden_layer = hidden_layer
        self.num_layers = num_layers
        
        # LSTM layers with dropout
        self.lstm = nn.LSTM(input_layer, hidden_layer, num_layers, batch_first=True, dropout=dropout)
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout)
        
        # Linear layer
        self.linear_layer = nn.Linear(hidden_layer, output_layer)

    def forward(self, input_tensor):
        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers, input_tensor.size(0), self.hidden_layer).to(input_tensor.device)
        c0 = torch.zeros(self.num_layers, input_tensor.size(0), self.hidden_layer).to(input_tensor.device)
        
        # Pass through LSTM layers
        out, _ = self.lstm(input_tensor, (h0, c0))
        
        # Apply dropout
        out = self.dropout(out)
        
        # Pass through linear layer
        out = self.linear_layer(out[:, -1, :])
        return out

# create sequences for input data and corresponding labels
def create_sequence(input_data, sequence_length):
    sequences = []
    for i in range(0, len(input_data) - sequence_length):
        sequence = input_data[i : i + sequence_length, :-1]
        label = input_data[i + sequence_length, -1]
        sequences.append((sequence, label))
    return sequences

# train the model with data provided
def trainer(model, train_data, loss_func, opt, epochs, fold, losses):
    model.train()
    for epoch in range(epochs):
        for sequence, labels in train_data:
            opt.zero_grad()
            sequence = sequence.clone().detach().float()
            labels = labels.clone().detach().float().view(-1, 1)
            
            y = model(sequence)
            loss = loss_func(y, labels)
            loss.backward()
            opt.step()

        if epoch % 25 == 1:
            print(f'Epoch {epoch} loss: {loss.item():.4f}')

            new_row = pd.DataFrame({'Fold': [fold+1], 'Epoch': [epoch], 'Loss': [loss.item()]})
            losses = pd.concat([losses, new_row], ignore_index=True)
            
    return losses

def predictor(model, test_data):
    model.eval()
    predictions = []
    with torch.no_grad():
        for sequence, _ in test_data:
            sequence = sequence.clone().detach().float()
            y = model(sequence)
            batch_predictions = torch.sigmoid(y)

            # Convert predictions to tensor if they are scalar or float
            if isinstance(batch_predictions, float) or batch_predictions.ndimension() == 0:
                batch_predictions = torch.tensor([batch_predictions])

            batch_predictions = torch.round(batch_predictions)

            predictions.extend(batch_predictions.squeeze().tolist())
    
    return predictions

# create sequences
sequence_length = 10
sequences = create_sequence(ticker[['Volume', 'CPI', 'Mortgage_rate', 'disposable_income','GVZ', 'OVX', 'VVIX', 'RSI (14D)', 
                                    '20 Day CCI', 'Williams %R', 'EMA (5D)', 'MA50', 'Target']].values, sequence_length)

# cross-validation
tscv = TimeSeriesSplit(n_splits=10)
accuracies = []
precisions = []
recalls = []
f1s = []
kappas = []
losses = pd.DataFrame(columns=['Fold', 'Epoch', 'Loss'])

for fold, (train_index, test_index) in enumerate(tscv.split(sequences)):
    print(f'Fold {fold+1}')
    
    train_sequences = [sequences[i] for i in train_index]
    test_sequences = [sequences[i] for i in test_index]
    
    train_data = torch.utils.data.DataLoader(train_sequences, shuffle=True, batch_size=32)
    test_data = torch.utils.data.DataLoader(test_sequences, shuffle=False, batch_size=32)
    
    # initialize model
    model = LSTM_Model(input_layer=12, hidden_layer=10, output_layer=1)
    loss_func = nn.BCEWithLogitsLoss()
    opt = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
    
    # train model
    epochs = 150
    losses = trainer(model, train_data, loss_func, opt, epochs, fold, losses)
    
    # make predictions
    test_labels = [label for _, label in test_sequences]
    predictions = predictor(model, test_data)
    
    # calculate statistics
    accuracy = accuracy_score(test_labels, predictions)
    precision = precision_score(test_labels, predictions, zero_division=0)
    recall = recall_score(test_labels, predictions)
    f1 = f1_score(test_labels, predictions)
    kappa = cohen_kappa_score(test_labels, predictions)
    
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)
    kappas.append(kappa)
    
    print(f'\nAccuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1: {f1:.4f}')
    print(f'Kappa: {kappa:.4f}\n')

# average scores across all folds
avg_accuracy = np.mean(accuracies)
avg_precision = np.mean(precisions)
avg_recall = np.mean(recalls)
avg_f1 = np.mean(f1s)
avg_kappa = np.mean(kappas)

print(f'\nCross-Validation Results:')
print(f'Average Accuracy: {avg_accuracy:.4f}')
print(f'Average Precision: {avg_precision:.4f}')
print(f'Average Recall: {avg_recall:.4f}')
print(f'Average F1: {avg_f1:.4f}')
print(f'Average Kappa: {avg_kappa:.4f}')


Fold 1
Epoch 1 loss: 0.6917
Epoch 26 loss: 0.6785
Epoch 51 loss: 0.6517
Epoch 76 loss: 0.5405
Epoch 101 loss: 0.5184
Epoch 126 loss: 0.5314

Accuracy: 0.4182
Precision: 0.4118
Recall: 0.5385
F1: 0.4667
Kappa: -0.1488

Fold 2
Epoch 1 loss: 0.7043
Epoch 26 loss: 0.6900
Epoch 51 loss: 0.6526
Epoch 76 loss: 0.6687
Epoch 101 loss: 0.6258
Epoch 126 loss: 0.5528

Accuracy: 0.5818
Precision: 1.0000
Recall: 0.0417
F1: 0.0800
Kappa: 0.0467

Fold 3
Epoch 1 loss: 0.6885
Epoch 26 loss: 0.6999
Epoch 51 loss: 0.5691
Epoch 76 loss: 0.5545
Epoch 101 loss: 0.4272
Epoch 126 loss: 0.8756

Accuracy: 0.5091
Precision: 0.5405
Recall: 0.6667
F1: 0.5970
Kappa: -0.0137

Fold 4
Epoch 1 loss: 0.7171
Epoch 26 loss: 0.6883
Epoch 51 loss: 0.6241
Epoch 76 loss: 0.5831
Epoch 101 loss: 0.6787
Epoch 126 loss: 0.6166

Accuracy: 0.5091
Precision: 1.0000
Recall: 0.1562
F1: 0.2703
Kappa: 0.1341

Fold 5
Epoch 1 loss: 0.6816
Epoch 26 loss: 0.6899
Epoch 51 loss: 0.6466
Epoch 76 loss: 0.6243
Epoch 101 loss: 0.7250
Epoch 126 los

In [77]:
fall = (ticker.Target == 0).sum()
rise = (ticker.Target == 1).sum()

rise_per = (rise / (rise + fall)) * 100
fall_per = (fall / (rise + fall)) * 100

print(f'Number of days rising: {rise}')
print(f'Number of days falling: {fall}')

print(f'Rise % is: {rise_per:.2f}%')
print(f'Fall % is: {fall_per:.2f}%')

Number of days rising: 322
Number of days falling: 296
Rise % is: 52.10%
Fall % is: 47.90%


In [80]:
results = pd.DataFrame(columns=['Dataset', 'Average Accuracy', 'Average Precision', 
                                'Average Recall', 'Average F1', 'Average Kappa', 'Number of Rises', 
                                'Number of Falls', 'Rise %', 'Fall %'])

datatset_name = (file_path.split('/')[-1].split('.')[0] + '_Monthly')

metrics = {'Dataset': datatset_name ,'Average Accuracy': avg_accuracy, 
           'Average Precision': avg_precision, 'Average Recall': avg_recall, 
           'Average F1': avg_f1, 'Average Kappa': avg_kappa, 'Number of Rises': rise, 'Number of Falls': fall, 
           'Rise %': rise_per, 'Fall %': fall_per}

export_results = results._append(metrics, ignore_index=True)

export_results = pd.concat([export_results, losses], ignore_index=True)

export_results.set_index('Dataset', inplace=True)

export_results.to_csv(r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\IRP DATA\\" + datatset_name + '_weekly_w_macro.csv')

export_results

,Average Accuracy,Average Precision,Average Recall,Average F1,Average Kappa,Number of Rises,Number of Falls,Rise %,Fall %,Fold,Epoch,Loss
Dataset,,,,,,,,,,,,
^NYA_Monthly,0.509091,0.517857,0.467263,0.397161,-0.010932,322,296,52.10356,47.89644,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0.691736
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,26,0.678465
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,51,0.651740
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,76,0.540539
...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,26,0.683795
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,51,0.795636
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,76,0.679115
